In [26]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

In [27]:
df = pd.read_parquet('../../../metrics.parquet')

In [28]:
df, _ = train_test_split(df, train_size=200000, stratify=df['time_to_stop_activity'], random_state=11)

In [29]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [30]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [31]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [32]:
def train_evaluate_catboost(n_estimators, learning_rate, max_depth, X_train, Y_train, X_test, Y_test):
    try:
        catboost = CatBoostRegressor(
            iterations=n_estimators,
            learning_rate=learning_rate,
            depth=max_depth,
            random_seed=11,
            verbose=0 
        )
        
        catboost.fit(X_train, Y_train)
        y_pred = catboost.predict(X_test)

        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)

        print(f"CatBoost - Estimators: {n_estimators}, Learning Rate: {learning_rate}, Max Depth: {max_depth} "
              f"Finalized - MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
        
        return {
            'n_estimators': n_estimators,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }
    
    except Exception as e:
        print(f"CatBoost - Estimators: {n_estimators}, Learning Rate: {learning_rate}, Max Depth: {max_depth} Error: {str(e)}")

        return {
            'n_estimators': n_estimators,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'mean_squared_error': None,
            'mean_absolute_error': None,
            'r2_score': None
        }

In [33]:
param_combinations = [
    (n_estimators, learning_rate, max_depth)
    for n_estimators in [1000, 400, 200, 100, 50]
    for learning_rate in [0.01, 0.1, 0.2, 0.4, 1]
    for max_depth in [3, 5, 10, 20, 40]
    ]

In [34]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [36]:
results = Parallel(n_jobs=-1)(
    delayed(train_evaluate_catboost)(n_estimators, lr, depth, X_train, Y_train, X_test, Y_test)
    for n_estimators, lr, depth in param_combinations
)

KeyboardInterrupt: 

In [12]:
benchmark_df = pd.DataFrame(results)

In [13]:
benchmark_df.to_excel('CatBoostRegressorBenchmark.xlsx')